In [1]:
# ТЕСТИРОВАНИЕ B20 RELEASE

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!rm -rf /content/drive/MyDrive/KIA_TEST/FAISS_KIA_B20
!mkdir -p /content/drive/MyDrive/KIA_TEST/FAISS_KIA_B20
!git clone -b faiss/B20 https://github.com/musicnova/KIA-GPT0.git FAISS_KIA_B20
!cp -r /content/FAISS_KIA_B20/knowledge/faiss_router /content/drive/MyDrive/KIA_TEST/FAISS_KIA_B20/faiss_router
!rm -r /content/FAISS_KIA_B20


Cloning into 'FAISS_KIA_B20'...
remote: Enumerating objects: 2118, done.
remote: Counting objects: 100% (531/531), done.
remote: Compressing objects: 100% (199/199), done.
remote: Total 2118 (delta 389), reused 420 (delta 332), pack-reused 1587
Receiving objects: 100% (2118/2118), 42.30 MiB | 33.04 MiB/s, done.
Resolving deltas: 100% (1233/1233), done.


In [3]:
!rm -rf faiss_routing
!cp -r /content/drive/MyDrive/KIA_TEST/FAISS_KIA_B20/faiss_router faiss_routing

# Code

In [4]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4 gspread oauth2client nltk pydantic==1.10.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 115.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 127.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not 

In [5]:
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
#database
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.document_loaders import TextLoader

from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken




In [6]:
import os
import platform

def clear_screen():
    system_platform = platform.system()  # определить операционную систему
    if system_platform == "Windows":
        os.system('cls')
    else:
        os.system('clear')

In [7]:
MODEL_GPT_3_5_TURBO_16K = ['gpt-3.5-turbo-16k', 0.003, 0.004]
MODEL_GPT_3_5_TURBO = ['gpt-3.5-turbo', 0.0015, 0.002]  # 4,097 tokens
MODEL_GPT_3_5_TURBO_INSTRUCT = ['gpt-3.5-turbo-instruct', 0.0015, 0.002]  # 4,097 tokens
MODEL_GPT_4 = ['gpt-4', 0.03, 0.06]  # 8,192 tokens
SELECT_MODEL_GPT = MODEL_GPT_4

In [8]:
# openai_key = getpass.getpass("OpenAI API Key:")
# os.environ["OPENAI_API_KEY"] = openai_key
# openai.api_key = openai_key

In [9]:
def insert_newlines(textstr: str, max_len: int = 170) -> str:
    words = textstr.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

In [10]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

## Собрать новую базу FAISS

In [11]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
import re

def load_file_knowledge(file_path: str) -> str:
    # Чтение текстового файла
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    headers_to_split_on = [
        ("#", "router"),
        ("##", "Header2"),
        ("###", "Header3"),
        ("####", "Header4"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(text)

    # Предполагается, что FAISS и OpenAIEmbeddings были импортированы или определены где-то выше
    vectordateBase = FAISS.from_documents(md_header_splits, OpenAIEmbeddings())

    return vectordateBase


In [12]:
## models_knowledge_base = load_file_knowledge('models.md')
## technology_knowledge_base = load_file_knowledge('technology.md')
## parts_knowledge_base = load_file_knowledge('parts.md')
## oils_knowledge_base = load_file_knowledge('oils.md')
## tech_knowledge_base = load_file_knowledge('tech.md')
## accessories_knowledge_base = load_file_knowledge('accessories.md')
## warranty_knowledge_base = load_file_knowledge('warranty.md')
## service_knowledge_base = load_file_knowledge('service.md')
## sales_knowledge_base = load_file_knowledge('sales.md')
## apps_knowledge_base = load_file_knowledge('apps.md')
## promotions_knowledge_base = load_file_knowledge('promotions.md')
## #none_knowledge_base = load_file_knowledge('none.md')

## Проверка чанков **БЕЗ** сбора нового файла FAISS **БЕЗ** обращения к OpenAI


In [13]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4 gspread oauth2client nltk pydantic==1.10.8

In [14]:
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
#database
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.document_loaders import TextLoader

from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


HISTORY = []

In [15]:
MODEL_GPT_3_5_TURBO_16K = ['gpt-3.5-turbo-16k', 0.003, 0.004]
MODEL_GPT_3_5_TURBO = ['gpt-3.5-turbo', 0.0015, 0.002]  # 4,097 tokens
MODEL_GPT_4 = ['gpt-4', 0.03, 0.06]  # 8,192 tokens
SELECT_MODEL_GPT = MODEL_GPT_3_5_TURBO_16K

In [16]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [17]:
# Загружаем векторный файл базы знаний! В коллаб надо в папку faiss загрузить 2 файла

models_knowledge_base = FAISS.load_local("faiss_routing/faiss_models", OpenAIEmbeddings())
technology_knowledge_base = FAISS.load_local("faiss_routing/faiss_technology", OpenAIEmbeddings())
parts_knowledge_base = FAISS.load_local("faiss_routing/faiss_parts", OpenAIEmbeddings())
oils_knowledge_base = FAISS.load_local("faiss_routing/faiss_oils", OpenAIEmbeddings())
tech_knowledge_base = FAISS.load_local("faiss_routing/faiss_tech", OpenAIEmbeddings())
accessories_knowledge_base = FAISS.load_local("faiss_routing/faiss_accessories", OpenAIEmbeddings())
warranty_knowledge_base = FAISS.load_local("faiss_routing/faiss_warranty", OpenAIEmbeddings())
service_knowledge_base = FAISS.load_local("faiss_routing/faiss_service", OpenAIEmbeddings())
sales_knowledge_base = FAISS.load_local("faiss_routing/faiss_sales", OpenAIEmbeddings())
apps_knowledge_base = FAISS.load_local("faiss_routing/faiss_apps", OpenAIEmbeddings())
promotions_knowledge_base = FAISS.load_local("faiss_routing/faiss_promotions", OpenAIEmbeddings())
#none_knowledge_base = FAISS.load_local("faiss_routing/faiss_models", OpenAIEmbeddings())

In [18]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate


models_template = """You are a very smart assistant for Kia. You know everything about KIA car models
You are excellent at answering questions about Kia car models concisely and clearly.
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{input}"""

technology_template = """You are a very smart assistant for KIA. You know everything about the technologies used in KIA cars.
You are excellent at answering questions briefly and clearly about the technologies used in KIA cars.
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{input}"""

parts_template = """You are a very smart assistant for KIA. You know all the information on spare parts for KIA cars.
You are excellent at answering questions about spare parts for KIA cars briefly and clearly.
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{input}"""

oils_template = """You are a very smart assistant for KIA. You know all the information on engine oils for KIA cars.
You are excellent at answering questions about engine oils for KIA cars briefly and clearly.
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{input}"""

tech_template = """You are a very smart assistant for KIA. You know everything about the maintenance of KIA cars.
You are excellent at answering questions about KIA vehicle maintenance briefly and clearly.
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{input}"""

accessories_template = """You are a very smart assistant for KIA. You know all the information about accessories for KIA cars.
You are excellent at answering questions about accessories for KIA cars concisely and clearly.
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{input}"""

warranty_template = """You are a very smart assistant for KIA. You know all the information about the warranty for KIA cars.
You are excellent at answering questions about KIA vehicle warranties concisely and clearly.
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{input}"""

service_template = """You are a very smart assistant for KIA. You know all the information about servicing KIA cars.
You are excellent at answering questions about service for KIA vehicles briefly and clearly.
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{input}"""

sales_template = """You are a very smart assistant for KIA. You know all the information about KIA car sales.
You are excellent at answering questions about KIA car sales briefly and clearly.
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{input}"""

apps_template = """You are a very smart assistant for KIA. You know all the information about KIA software.
You are excellent at answering questions about KIA software briefly and clearly.
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{input}"""

promotion_template = """You are a very smart assistant for KIA. You know everything about promotions of KIA company.
You are excellent at answering questions about promotions of KIA company concisely and clearly.
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{input}"""



prompt_infos = [
    {
        "name": "models",
        "description": "Good for answering questions about models kia auto",
        "prompt_template": models_template,
    },
    {
        "name": "technology",
        "description": "Good for answering questions about Kia technology.",
        "prompt_template": technology_template,
    },
    {
        "name": "parts",
        "description": "Good for answering questions about Kia parts.",
        "prompt_template": parts_template,
    },
    {
        "name": "oils",
        "description": "Good for answering questions about Kia engine oils.",
        "prompt_template": oils_template,
    },
    {
        "name": "tech",
        "description": "Good for answering questions about technical problems with Kia vehicles.",
        "prompt_template": tech_template,
    },
    {
        "name": "accessories",
        "description": "Good for answering questions about accessories used in Kia vehicles.",
        "prompt_template": accessories_template,
    },
    {
        "name": "warranty",
        "description": "Good for answering questions about Kia vehicle warranties.",
        "prompt_template": warranty_template,
    },
    {
        "name": "service",
        "description": "Good for answering Kia vehicle service questions.",
        "prompt_template": service_template,
    },
    {
        "name": "sales",
        "description": "Good for answering Kia car sales questions.",
        "prompt_template": sales_template,
    },
    {
        "name": "apps",
        "description": "Good for answering questions about using Kia software.",
        "prompt_template": apps_template,
    },
    {
        "name": "promotion",
        "description": "Good for answering questions about Kia promotions.",
        "prompt_template": promotion_template,
    },
]

llm = OpenAI()

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)


router_chain = LLMRouterChain.from_llm(llm, router_prompt)



chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)


models: Good for answering questions about models kia auto
technology: Good for answering questions about Kia technology.
parts: Good for answering questions about Kia parts.
oils: Good for answering questions about Kia engine oils.
tech: Good for answering questions about technical problems with Kia vehicles.
accessories: Good for answering questions about accessories used in Kia vehicles.
warranty: Good for answering questions about Kia vehicle warranties.
service: Good for answering Kia vehicle service questions.
sales: Good for answering Kia car sales questions.
apps: Good for answering questions about using Kia software.
promotion: Good for answering questions about Kia promotions.


In [19]:
def _summarize_topic( dialog):
        messages = [
            {"role": "system",
             "content": "Ты - ассистент консультанта, основанный на AI. Ты умеешь профессионально суммаризировать присланные тебе диалоги консультанта и клиента. Твоя задача - суммаризировать диалог, который тебе пришел."},
            {"role": "user",
             "content": "Суммаризируй следующий диалог консультанта и клиента: " + " ".join(dialog)}
        ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=0.1,  # Используем более низкую температуру для более определенной суммаризации
            max_tokens=1000  # Ограничиваем количество токенов для суммаризации
        )

        return completion.choices[0].message.content

def answer_index(topic, temp=0.1, top_similar_documents=3):

        router_result = router_chain.__call__({"input": topic})
        chosen_route = router_result['destination']
        next_inputs = router_result['next_inputs']
        print(f"Выбранный маршрут: {chosen_route}")
        print(f"Следующие входные данные: {next_inputs}")

        unit_to_multiplier = {
            'models':  models_knowledge_base,
            'technology': technology_knowledge_base,
            'parts':  parts_knowledge_base,
            'oils': oils_knowledge_base,
            'tech': tech_knowledge_base,
            'accessories': accessories_knowledge_base,
            'warranty': warranty_knowledge_base,
            'service': service_knowledge_base,
            'sales': sales_knowledge_base,
            'apps': apps_knowledge_base,
            'promotions': promotions_knowledge_base
        }

        summarize_text = ''
        if len(HISTORY) > 0:
            summarize_text = "Вот краткий обзор предыдущего диалога: " + _summarize_topic(
                [q + ' ' + (a if a is not None else '') for q, a in HISTORY])
            print(f'САММАРИ \n=== {summarize_text} \n')

        # Добавляем явное разделение между историей диалога и текущим вопросом
        input_text = summarize_text + "\n\nТекущий вопрос: " + topic

        knowledge_base = unit_to_multiplier.get(chosen_route, None)
        if knowledge_base is not None:
            docs = knowledge_base.similarity_search_with_score(topic, k=top_similar_documents)
        else:
            return  insert_newlines('Вопрос не понятен!!!')


        responses = []
        for i, (doc, score) in enumerate(docs):
            if score < 1: # ТУТ ТЫ МОЖЕШЬ УПРАВЛЯТЬ праметром Л2 для чанков. 0..1
                content = doc.page_content
                response = f'\n=======Отрывок документа №{i + 1}=====\n{content}\n'
                print(f'\n=====================Отрывок документа №{i + 1}=====================\n')
                print(f'=== score = {score}  Metadata документа ------------ {doc.metadata}')
                print(f'\n{content}\n')
                responses.append(response)

        messages = [
            {"role": "system", "content": prompt},
            {"role": "user",
             "content": f"Документ с информацией для ответа клиенту: {responses}\n\nВопрос клиента: \n{input_text}"}
        ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=temp
        )

        answer = completion.choices[0].message.content

        # Добавляем вопрос пользователя и ответ системы в историю
        HISTORY.append((topic, answer if answer is not None else ''))

        return  insert_newlines(answer)


In [20]:
def run_dialog():
    import time
    HISTORY = []
    AUTO_HELLO="Иван. Мне 10 лет. У папы сломалась KIA RIO в стандартной комплектации."
    topic = 'Приветствую, я нейро-помощник KIA. Подскажите, какая у Вас марка машины? Как я могу к Вам обращаться?'
    print('\nМенеджер: ', topic+'\n\n')
    print('\nКлиент: ', AUTO_HELLO+'\n\n')
    user_question = AUTO_HELLO
    answer = answer_index(user_question)
    HISTORY.append((topic, answer if answer is not None else ''))
    print('\nМенеджер: ', 'Какая Вам требуется помощь?'+'\n\n')
    while True:
        user_question = input('\nКлиент: ')
        if ((user_question.lower() == 'stop') or (user_question.lower() == 'стоп')):
            break
        try:
            answer = answer_index(user_question)
        except:
            time.sleep(1)
            HISTORY = []
            answer_index(None, None, None, knowledge_only_langchain, AUTO_HELLO)
        print('\nМенеджер: ', answer+'\n\n')

    return

In [21]:
# Промпт моделей машин
prompt = load_document_text ('https://docs.google.com/document/d/1i8HA7cX4Ut-tb9rf8wOgERU7lLe66xJYscizGtSSJl0')

In [22]:

run_dialog()


Менеджер:  Приветствую, я нейро-помощник KIA. Подскажите, какая у Вас марка машины? Как я могу к Вам обращаться?



Клиент:  Иван. Мне 10 лет. У папы сломалась KIA RIO в стандартной комплектации.




/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Выбранный маршрут: service
Следующие входные данные: {'input': 'Папа имеет проблему с KIA RIO в стандартной комплектации. Какой вид сервисной поддержки необходим?'}

=====================Отрывок документа №1=====================

=== score = 0.2921800911426544  Metadata документа ------------ {'router': 'service', 'Header2': 'Рекомендация обратиться в службу "Помощь на дороге" для проблем с Kia Рио'}

В случае проблем с заводом автомобиля Киа Рио, рекомендуется обратиться в службу "Помощь на дороге" по номеру 8 800 301 08 80.


=====================Отрывок документа №2=====================

=== score = 0.3013317584991455  Metadata документа ------------ {'router': 'service', 'Header2': 'Проблема с коробкой передач и чеком на Kia Rio'}

В случае проблем с коробкой передач и загоревшимся чеком на Киа Рио, необходимо обратиться в дилерский центр. Список официальных дилеров Kia доступен на сайте компании.


=====================Отрывок документа №3=====================

=== score = 0.30305